# Differential Evolution

**Equations and pseudo code is taken from "Learning Algorithms and Neural Networks" notes written by Dr. Victor Parque.**

## Basic Differential Evolution Algorithm

For a population of $NP$ vectors $x_i$, each in $\mathbb{R}^D$, three randomly sampled vectors $x_{r1}$, $x_{r2}$ and $x_{r3}$, a mutant individual $v_t$ is calculated as follows:
$$
v = x_{r1} + F(x_{r2} - x_{r3})
$$
where $F$ is a scaling factor $\in [0, 1]$.

Then, a mutation vector $u$ is caluclated by randomly mutating random entries in $v$ as follows:

$$
e_{j} = 0\ if\ Rand \leq CR \ or\ j = Rand;\ 0\ otherwise
$$
$$
e = [e_1, e_2, \dots , e_D]
$$

Then

$$
u_t = x_i + e \dot (v - x_i)
$$

Finally,

$$
x_i^{new} = u \ if \ f(u) < f(x_i) \ ;\ 0\ otherwise.
$$

Algorithm Pseudo code:

```python
Generate initial population
Repeat:
    for each individual in population:
        randomly select x_r1, x_r2, x_r3 from the population
        Compute mutant v
        Compute trial u
        if f(u) < f(x):
            x = u
        
Until termination criteria ends
```

In [1]:
import numpy as np
from matplotlib import pyplot as plt